# Basic Thicket Tutorial: Thicket 101

Thicket is a python-based toolkit for Exploratory Data Analysis (EDA) of parallel performance data that enables performance optimization and understanding of applications’ performance on supercomputers. It bridges the performance tool gap between being able to consider only a single instance of a simulation run (e.g., single platform, single measurement tool, or single scale) and finding actionable insights in multi-dimensional, multi-scale, multi-architecture, and multi-tool performance datasets.

**NOTE: An interactive version of this notebook is available in the Binder environment.**

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/llnl/thicket-tutorial/develop)

***

## 1. Import Necessary Packages

To explore the structure and various capabilities of thicket components, we begin by importing necessary packages. These include python extensions and thicket's statistical functions.

In [ ]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import HTML
import hatchet as ht

import thicket as tt

display(HTML("<style>.container { width:80% !important; }</style>"))

## 2. Read in Performance Profiles

For this notebook, we select profiles generated on Lawrence Livermore National Lab (LLNL) machine, lassen. We create two thicket objects, one generated with the same block size of 128 and the other generated with different block sizes (128 and 256).   

In [ ]:
problem_sizes = [
    "1048576", 
    "2097152", 
    "4194304", 
    "8388608"
]
lassen1 = [f"../data/lassen/clang10.0.1_nvcc10.2.89_{x}/Base_CUDA-block_128.cali" for x in problem_sizes]
lassen2 = [f"../data/lassen/clang10.0.1_nvcc10.2.89_1048576/Base_CUDA-block_256.cali"]

# generate thicket(s)
th_lassen = tt.Thicket.from_caliperreader(lassen1)
th_obj = tt.Thicket.from_caliperreader(lassen1 + lassen2)

## 3. More Information on a Function

You can use the `help()` method within Python to see the information for a given object. You can do this by typing `help(object)`. 
This will allow you to see the arguments for the function, and what will be returned. An example is below.

In [ ]:
help(tt.median)

## 4. Thicket Components

### 4.1 Performance Data

The performance data table is a multi-dimensional, multi-indexed component of thicket. The rows represent nodes that each contain a different execution (i.e., profile index) of the associated call tree node. 


#### View performance data table

In [ ]:
display(HTML(th_lassen.dataframe.to_html()))

#### Composing multiple thickets
We can compose thickets in a hierarchical, horizontal ordering using thicket's `columnar_join` function. In this example, we compose profiles of four different problem sizes and four different block sizes seamlessly.

In [ ]:
data = {
    "block_128": [f"../data/lassen/clang10.0.1_nvcc10.2.89_{x}/Base_CUDA-block_128.cali" for x in problem_sizes],
    "block_256": [f"../data/lassen/clang10.0.1_nvcc10.2.89_{x}/Base_CUDA-block_256.cali" for x in problem_sizes],
    "block_512": [f"../data/lassen/clang10.0.1_nvcc10.2.89_{x}/Base_CUDA-block_512.cali" for x in problem_sizes],
    "block_1024": [f"../data/lassen/clang10.0.1_nvcc10.2.89_{x}/Base_CUDA-block_1024.cali" for x in problem_sizes],
}

In [ ]:
block_128 = tt.Thicket.from_caliperreader(data["block_128"])
block_256 = tt.Thicket.from_caliperreader(data["block_256"])
block_512 = tt.Thicket.from_caliperreader(data["block_512"])
block_1024 = tt.Thicket.from_caliperreader(data["block_1024"])

In [ ]:
th_cj = tt.Thicket.concat_thickets(
    axis="columns",
    thickets=[block_128, block_256, block_512, block_1024],
    headers=["Block 128", "Block 256", "Block 512", "Block 1024"],
    metadata_key="ProblemSizeRunParam"
)

#### View combined thicket's performance data

In [ ]:
display(HTML(th_cj.dataframe.to_html()))

### 4.2 Metadata

The metadata table stores HPC simulation information such as an application’s build settings and execution context. A row corresponds to a single execution of the application and is identified by a unique profile index.

#### View metadata table:

In [ ]:
display(HTML(th_lassen.metadata.to_html()))

#### Filter with respect to metadata

The metadata table of a thicket helps select certain variation based on specific metadata. For example, selecting a certain compiler. In this example, we filter the metadata to select the profiles generated with a problem size of `1048576`.

In [ ]:
# selecting profiles with specified problem size
filter_metadata_func = lambda x: x["ProblemSizeRunParam"] == 1048576.0
th_example = th_obj.filter_metadata(filter_metadata_func)
display(HTML(th_example.metadata.to_html()))

#### Groupby using the metadata 

The metadata table also supports the grouping of the thicket based on the unique values present in the provided column(s). The following example groups the thicket according to any unique combination of values in the `launchdate` and `ProblemSizeRunParam` columns.

In [ ]:
# create sub-thickets from unique values in two columns
grouping_metadata_cols = ["launchdate", "ProblemSizeRunParam"]
sub_thickets = th_lassen.groupby(grouping_metadata_cols)
for groupby_cols, thicket in sub_thickets.items():
    display(thicket.metadata)

### 4.3 Aggregated Statistics

The aggregated statistics in a thicket is a GraphFrame. Therefore, it contains a graph and corresponding dataframe component. The table supports an order-reduction mechanism and stores processed applications’ performance. Each row of the aggregated statistics table holds data aggregated across all profiles associated with a particular call tree node. Below is an example of an empty aggregated statistics table 


#### View aggregated statistics table

In [ ]:
display(HTML(th_lassen.statsframe.dataframe.to_html()))

#### Filter with respect to aggregated statistics

The aggregated statistics table also supports a filter function. In the example below, we filter the table to select the nodes with the names `RAJAPerf`, `Algorithm`, and `Stream_MUL`.

In [ ]:
stats_nodes = ["RAJAPerf", "Algorithm", "Stream_MUL"]
th_stats_name = th_obj.filter_stats(lambda x: x["name"] in stats_nodes)
display(HTML(th_stats_name.statsframe.dataframe.to_html()))

#### Calculate the median and mean of performance data column, append to aggregated statistics table

The aggregated statistics table allows users to select a column from the performance data to perform an average on the values in a column. After performing the `median()` and `mean()` operation on said column, two new columns are appended to the statistics table with the mean and median values corresponding to the columns provided by the user. 

Below is an example where we calculate the mean and median of the values in the `Total time` column, which is the total exclusive time corresponding to the respective nodes.  

In [ ]:
metrics = ["Total time"]
tt.median(th_lassen, columns=metrics)
display(HTML(th_lassen.statsframe.dataframe.to_html()))

In [ ]:
tt.mean(th_lassen, columns=metrics)
display(HTML(th_lassen.statsframe.dataframe.to_html()))

#### Calculate the percentile of performance data column, append to aggregated statistics table

The aggregated statistics table allows users to select a column from the performance data to perform the `percentiles()` operation. This results in a new column appended to the statistics table containing the 25th, 50th, and 75th percentiles of the values in the provided column.

Below is an example where we calculate the percentiles of the values in the same `Total time` column.

In [ ]:
tt.percentiles(th_lassen, columns=metrics)
display(HTML(th_lassen.statsframe.dataframe.to_html()))

#### View aggregated statistics call tree

In [ ]:
print(th_lassen.statsframe.tree(metric_column="Total time_median"))

#### Use the query language

Thicket's query language provides users the capability to select or `query` specific nodes based on the call tree of the thicket. The performance data is then updated as part of the operation. 

**Initial call tree:** 

In [ ]:
print(th_lassen.statsframe.tree("Total time_median"))

**Example 1**

In the example below, we use a thicket query that only select the parents and children nodes of `Algorithm`, maintaining the structure of the call tree.

NOTE: A `DeprecationWarning` is generated when using “old-style” queries (i.e., queries with `QueryMatcher`) when using the newest version of Hatchet. In the future, queries will be defined with `Query`.

In [ ]:
alg_query_ex1 = (
    ht.QueryMatcher()
    .match("*")
    .rel(
        ".", 
        lambda row: row["name"].apply(
        lambda x: re.match(
            r"Algorithm.*", x
        ) 
        is not None).all()
    )
)
    
# applying the query on the lassen thicket
th_algorithm_ex1 = th_lassen.query(alg_query_ex1)
tt.median(th_algorithm_ex1, columns=["Total time"])
print(th_algorithm_ex1.statsframe.tree("Total time_median"))

**Example 2**

In the example below, we use a thicket query that selects only the `Algorithm` node and its children nodes, maintaining the structure of the call tree.

NOTE: A `DeprecationWarning` is generated when using “old-style” queries (i.e., queries with `QueryMatcher`) when using the newest version of Hatchet. In the future, queries will be defined with `Query`.

In [ ]:
alg_query_ex2 = (
    ht.QueryMatcher()
    .match(
        ".",
        lambda row: row["name"].apply(
        lambda x: re.match(
            r"Algorithm.*", x
        ) 
        is not None).all()
    )
    .rel("*")
)

# applying the second query on the lassen thicket
th_algorithm_ex2 = th_lassen.query(alg_query_ex2)
tt.median(th_algorithm_ex2, columns=["Total time"])
print(th_algorithm_ex2.statsframe.tree("Total time_median"))

#### Display histogram

The `display_histogram()` function allows users to select a node and metric value (a column in the performance data table) for which a histogram is generated.

Some available keyword arguments are the following,

* `height`: height (in inches) of each facet.
* `aspect`: aspect ratio of each facet, aspect * height will give you the width of each facet.
* `bins`: rather generic, and can set the number of bins.
* `binwidth`: width of each bin, overrides bins but can be used with binrange.
* `binrange`: lowest and highest value for bin edges; can be used either with bins or binwidth. Defaults to extremes.
* `color`: Set the color of the bars.

An exhaustive list of available arguments can be found [here](https://seaborn.pydata.org/generated/seaborn.displot.html).   

In [ ]:
n = pd.unique(th_algorithm_ex1.dataframe.reset_index()["node"])[0]

In [ ]:
tt.display_histogram(th_algorithm_ex1, node=n, column="Total time")

#### Display heatmap

The `display_heatmap()` function allows users to select column(s) from the performance data table, for which a heatmap is generated based on the values of the column.

Some available keyword arguments are the following: 
* `vmax`: maximum value to anchor the color map.
* `vmin`: minimum value to anchor the color map.
* `linecolor`: color of the lines that will divide each cell.
* `linewidths`: Width of the lines that will divide each cell.

An exhaustive list of available arguments can be found [here](https://seaborn.pydata.org/generated/seaborn.heatmap.html).  

In [ ]:
plt.figure(figsize=(30, 30))
metrics = ["Total time_median"]
tt.display_heatmap(th_algorithm_ex1, columns=metrics)